In [3]:
#load libraries
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
pd.set_option('display.float_format', lambda x: '%.3f' % x)


#Data profiling/EDA 
import pandas_profiling 

import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

In [4]:
out_path = str(Path.cwd().parent)+ '/input/'

os.makedirs(os.path.dirname(out_path), exist_ok=True)

In [56]:
dftrain = pd.read_csv('{}/titanic/train.csv'.format(out_path))
dftest = pd.read_csv('{}/titanic/test.csv'.format(out_path))


In [57]:
dftest['Survived'] = -999

In [58]:
dftest =dftest[dftrain.columns]

In [59]:
df = pd.concat([dftrain,dftest])

In [60]:
df.columns = df.columns.str.lower()


In [61]:
df.head()

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


In [62]:
#rename the target var
df['target'] = df['survived']

df.drop(['survived'],axis = 1, inplace = True)

# df['target'] = pd.to_numeric(df.target)


In [63]:
#rename the target var
df['id'] = df['passengerid']

df.drop(['passengerid'],axis = 1, inplace = True)

In [64]:
df.drop(['name'],axis = 1, inplace = True)


In [66]:
df.replace(np.nan,-1,inplace = True)

In [67]:
df[df.target != -999].to_csv('{}/titanic/train.csv'.format(out_path), index = False)

In [68]:

df[df.target == -999].to_csv('{}/titanic/test.csv'.format(out_path), index = False)


In [69]:
df = pd.read_csv('{}/titanic/train_folds.csv'.format(out_path))


In [70]:
df.kfold.unique()

array([0, 1, 2, 3, 4])

In [71]:
df.head()

,pclass,sex,age,sibsp,parch,ticket,fare,cabin,embarked,target,id,kfold
0,3,male,22.000,1,0,A/5 21171,7.250,-1,S,0,1,0
1,1,female,38.000,1,0,PC 17599,71.283,C85,C,1,2,0
2,3,female,26.000,0,0,STON/O2. 3101282,7.925,-1,S,1,3,0
3,1,female,35.000,1,0,113803,53.100,C123,S,1,4,0
4,3,male,35.000,0,0,373450,8.050,-1,S,0,5,0


In [72]:
df.dtypes

pclass        int64
sex          object
age         float64
sibsp         int64
parch         int64
ticket       object
fare        float64
cabin        object
embarked     object
target        int64
id            int64
kfold         int64
dtype: object

In [73]:
df.isnull().sum()

pclass      0
sex         0
age         0
sibsp       0
parch       0
ticket      0
fare        0
cabin       0
embarked    0
target      0
id          0
kfold       0
dtype: int64

In [74]:
from sklearn import preprocessing


cat_features = list(df.select_dtypes(include=["object"]).columns)
print("Categorical: {} features".format(len(cat_features)))

print(cat_features)

# in next iteration need to handel cat intelligently
label_encoders = {}
for c in cat_features:
    lbl = preprocessing.LabelEncoder()
    df.loc[:, c] = df.loc[:, c].astype(str).fillna("NONE")
    # valid_df.loc[:, c] = valid_df.loc[:, c].astype(str).fillna("NONE")
    # df_test.loc[:, c] = df_test.loc[:, c].astype(str).fillna("NONE")
    lbl.fit(
        df[c].values.tolist()
         
    )
    df.loc[:, c] = lbl.transform(df[c].values.tolist())
    # valid_df.loc[:, c] = lbl.transform(valid_df[c].values.tolist())
    label_encoders[c] = lbl

Categorical: 4 features
['sex', 'ticket', 'cabin', 'embarked']


In [75]:
df.loc[:, c].astype(str).fillna("NONE")

0      3
1      1
2      3
3      3
4      3
      ..
886    3
887    3
888    3
889    1
890    2
Name: embarked, Length: 891, dtype: object